<a href="https://colab.research.google.com/github/dionisispriftis/Saliency-Maps/blob/main/C3W4_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Week 4 Assignment: Saliency Maps**

A saliency map shows the pixels which greatly impacts the classification of an image. 
- This is done by getting the gradient of the loss with respect to changes in the pixel values, then plotting the results. 
- From there, you can see if your model is looking at the correct features when classifying an image. 
  - For example, if you're building a dog breed classifier, you should be wary if your saliency map shows strong pixels outside the dog itself (e.g. sky, grass, dog house, etc...).




### Download test files and weights



In [ ]:
# Download the same test files from the Cats vs Dogs ungraded lab
!wget -O cat1.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/cat1.jpg
!wget -O cat2.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/cat2.jpg
!wget -O catanddog.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/catanddog.jpg
!wget -O dog1.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/dog1.jpg
!wget -O dog2.jpg https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/dog2.jpg

# Download prepared weights
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1kipXTxesGJKGY1B8uSPRvxROgOH90fih' -O 0_epochs.h5
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1oiV6tjy5k7h9OHGTQaf0Ohn3FmF-uOs1' -O 15_epochs.h5
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=14vFpBJsL_TNQeugX8vUTv8dYZxn__fQY' -O 95_epochs.h5

### Import the required packages


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers
from tensorflow.keras import Model
import cv2

### Download and prepare the dataset.



#### Load Cats vs Dogs 


In [ ]:
# Load the data and create the train set (optional: val and test sets)

train_ds = tfds.load('cats_vs_dogs', split='train[:80%]', as_supervised = True)
validation_ds = tfds.load('cats_vs_dogs', split='train[80%:90%]', as_supervised = True)
test_ds = tfds.load('cats_vs_dogs', split='train[90%:]', as_supervised = True)

#### Create preprocessing function

This function will:
  * cast the image to float32
  * normalize the pixel values to [0, 1]
  * resize the image to 300 x 300


In [ ]:
def augment_train_images(image, label):
  
  image = tf.cast(image, tf.float32)
  image /= 255.0
  image = tf.image.resize(image, (300, 300))

  return image, label

def augment_test_images(image, label):
  
  image = tf.cast(image, tf.float32)
  image /= 255.0
  image = tf.image.resize(image, (300, 300))
  image = tf.expand_dims(image, axis=0)
  label = tf.expand_dims(label, axis=0)
  print(label.shape)

  return image, label 

#### Preprocess the training set

We use the `map()` and pass in the method that we just defined to preprocess the training set.


In [ ]:
augmented_training_data = train_ds.map(augment_train_images)
augmented_validation_data = validation_ds.map(augment_test_images)
augmented_test_data = test_ds.map(augment_test_images)

#### Create batches of the training set. 



In [ ]:
train_batches = augmented_training_data.batch(32)

### Build the Cats vs Dogs classifier 



In [ ]:
def conv_block(input, filters, global_pooling = False):

  x = Conv2D(filters=filters,kernel_size=(3,3),activation='relu',padding='same')(input)
  if(global_pooling):
    return GlobalAveragePooling2D()(x)
  else:
    return MaxPooling2D(pool_size=(2,2))(x)


In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D

input = tf.keras.Input(shape=(300, 300, 3))
x = conv_block(input, 16)
x = conv_block(x, 32)
x = conv_block(x, 64)
x = conv_block(x, 128, True)
output = Dense(2,activation='sigmoid')(x)

model = Model(inputs = input, outputs = output)
model.summary()

### Create a function to generate the saliency map


In [ ]:
def do_salience(image, model, label, prefix):
  '''
  Generates the saliency map of a given image.

  Args:
    image (file) -- picture that the model will classify
    model (keras Model) -- your cats and dogs classifier
    label (int) -- ground truth label of the image
    prefix (string) -- prefix to add to the filename of the saliency map
  '''

  # Read the image and convert channel order from BGR to RGB
  TEST_IMAGES_PATH = '/content'
  num_classes = 2


  img = cv2.imread(TEST_IMAGES_PATH + '/' + image )
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img, (300, 300)) / 255

  # Add an additional dimension (for the batch), and save this in a new variable
  tensor_image = tf.expand_dims(img, axis = 0)
  
  # Define the expected output array by one-hot encoding the label
  # The length of the array is equal to the number of classes
  expected_output = tf.one_hot([label] * tensor_image.shape[0], num_classes)
  tensor_image = tf.cast(tensor_image, tf.float32)
  
  with tf.GradientTape() as tape:
    tape.watch(tensor_image)
    y_pred = model(tensor_image)
    loss = tf.keras.losses.categorical_crossentropy(y_pred, expected_output)
    print('prediction:', y_pred), print('loss:', loss)
    
  gradients = tape.gradient(loss, tensor_image)
    
  # generate the grayscale tensor
  grayscale_tensor = tf.reduce_sum(tf.abs(gradients), axis=-1)

  # normalize the pixel values to be in the range [0, 255].
  # Cast the tensor as a tf.uint8

  max, min = tf.reduce_max(grayscale_tensor), tf.reduce_min(grayscale_tensor)
  normalized_tensor = 255 * (grayscale_tensor - min) / (max - min)
  normalized_tensor = tf.cast(normalized_tensor, tf.uint8)  

  # Remove dimensions that are size 1
  normalized_tensor = tf.squeeze(normalized_tensor)
    
  # plot the normalized tensor and the superimposed saliency map to the original image  
  fig, ax = plt.subplots(1, 2, figsize=(16,8))
  
  gradient_color = cv2.applyColorMap(normalized_tensor.numpy(), cv2.COLORMAP_HOT)
  gradient_color = gradient_color / 255.0
  super_imposed = cv2.addWeighted(img, 0.5, gradient_color, 0.5, 0.0)
  
  ax[0].axis('off'), ax[1].axis('off')
  ax[0].imshow(normalized_tensor, cmap='gray'), ax[1].imshow(super_imposed)
  plt.show()

  # save the normalized tensor image to a file.
  salient_image_name = prefix + image
  normalized_tensor = tf.expand_dims(normalized_tensor, -1)
  normalized_tensor = tf.io.encode_jpeg(normalized_tensor, quality=100, format='grayscale')
  writer = tf.io.write_file(salient_image_name, normalized_tensor)

### Configure the model for training



In [ ]:
opt = tf.keras.optimizers.RMSprop()
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=opt,
              loss=loss, 
              metrics=['accuracy'])

### Train your model


In [ ]:
def generate_saliency(filenames, prefix):

  for name in filenames:
  label = 1
  if name[0:3]=='cat':
    label = 0
  print(name)
  do_salience(name, model, label, prefix)

In [ ]:
FILENAMES = ['cat1.jpg','cat2.jpg','catanddog.jpg','dog1.jpg','dog2.jpg']
EPOCHS = 10

# load pre-trained weights
model.load_weights('15_epochs.h5')
model.fit(train_batches, epochs=3, validation_data=augmented_validation_data)
generate_saliency(FILENAMES, 'epoch25_salient')

In [ ]:
model.load_weights('95_epochs.h5')

# generate the saliency maps for the 5 test images
generate_saliency(FILENAMES, 'epoch95_salient')